In [1]:
import numpy as np
import pandas as pd
import time
import multiprocessing
from multiprocessing import Pool
import functools
import sys
from pfsm import *

In [2]:
pd.set_option('display.max_colwidth', -1)
dtf_names =  pd.read_csv('data/sec_edgar_company_info.csv', dtype = object)
print(dtf_names.shape)
dtf_names.head()

(663000, 3)


,Line Number,Company Name,Company CIK Key
0,1,!J INC,1438823
1,2,"#1 A LIFESAFER HOLDINGS, INC.",1509607
2,3,#1 ARIZONA DISCOUNT PROPERTIES LLC,1457512
3,4,#1 PAINTBALL CORP,1433777
4,5,$ LLC,1427189


In [3]:
time_start = time.time()
str_strings = dtf_names["Company Name"].unique()#[0:300000]
M = tfidfM(str_strings)
print(M.shape)
print(time.time() - time_start)

(657160, 29619)
56.56030893325806


In [4]:
def myf(idxs, ns):
    min_idxs = min(idxs)
    matches_tmp = awesome_cossim_top(ns.M[idxs, ], ns.M[min_idxs:,].transpose(), 10, 0.85)
    return [matches_tmp[j, ].indices + min_idxs for j in range(matches_tmp.shape[0])]

In [5]:
time_start = time.time()
int_entity_id = np.array([np.NaN for j in range(len(str_strings))])
lst_split = list(split(range(M.shape[0]), 500))
print(time.time() - time_start)

0.0826566219329834


In [6]:
time_start = time.time()
manager = multiprocessing.Manager()
ns = manager.Namespace()
ns.M = M
print(time.time() - time_start)

0.5665156841278076


In [7]:
time_start = time.time()
pool = Pool(10)
r = pool.map(functools.partial(myf, ns = ns), lst_split)
pool.close()
pool.join()
print(time.time() - time_start)

669.5270597934723


In [8]:
time_start = time.time()


out = np.array([])
for j in range(len(r)):
    out = np.append(out, r[j])

out
r = out

    
for j in range(len(r)):
    idx = r[j]
    if np.isscalar(idx):
        idx = np.array([idx])
    idx = idx.astype(int)
    tmp_idx = int_entity_id[idx]
    if sum([np.isnan(x) for x in tmp_idx]) == len(tmp_idx):
        int_entity_id[idx] = j
    else:
        int_entity_id[idx] = np.nanmin(tmp_idx)


dtf_out = pd.DataFrame({"STRING": str_strings, "ENTITY_ID": int_entity_id}).sort_values(["ENTITY_ID"])
print(dtf_out.shape)


print(time.time() - time_start)

(657160, 2)
22.526001691818237


In [9]:
dtf_out[dtf_out.ENTITY_ID.isnull()].head()

,STRING,ENTITY_ID


In [10]:
tmp = dtf_out.groupby(["ENTITY_ID"]).count().reset_index()
tmp.columns = ["ENTITY_ID", "COUNT"]
dtf_out = pd.merge(dtf_out, tmp, on = "ENTITY_ID", how = "left").drop_duplicates()
print(dtf_out.shape)

(657160, 3)


In [11]:
idx = 0
dtf_out.sort_values(["COUNT", "ENTITY_ID"], ascending = False)[(idx):(idx+20)]

,STRING,ENTITY_ID,COUNT
161089,DEFINED ASSET FUNDS MUNI INV TR FD MO PAYMENT SERIES 1,168040.0,1447
161090,DEFINED ASSET FUNDS GOVERNMENT SECURITIES INC FD GNMA SER 1A,168040.0,1447
161091,DEFINED ASSET FUNDS GOVERNMENT SECURITIES INC FD GNMA SER 19,168040.0,1447
161092,DEFINED ASSET FUNDS GOVERNMENT SECURITIES INC FD GNMA SER 18,168040.0,1447
161093,DEFINED ASSET FUNDS GOVERNMENT SECURITIES INC FD GNMA SER 17,168040.0,1447
161094,DEFINED ASSET FUNDS GOVERNMENT SECURITIES INC FD GNMA SER 16,168040.0,1447
161095,DEFINED ASSET FUNDS GOVERNMENT SECURITIES INC FD GNMA SER 15,168040.0,1447
161096,DEFINED ASSET FUNDS GOVERNMENT SECURITIES INC FD GNMA SER 14,168040.0,1447
161097,DEFINED ASSET FUNDS GOVERNMENT SECURITIES INC FD GNMA SER 13,168040.0,1447
161098,DEFINED ASSET FUNDS GOVERNMENT SECURITIES INC FD GNMA SER 12,168040.0,1447
